In [5]:
# Libraries
import altair as alt
import pandas as pd

In [174]:
# connect to bigquery
import json
from google.oauth2 import service_account
from google.cloud import bigquery

path = '***'

with open(path, 'r') as f:
    json_key_file = json.load(f)
    
credentials = service_account.Credentials.from_service_account_info(json_key_file)

print(credentials)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)
datasets = list(client.list_datasets())

print(f'Datasets in project {client.project}:')

for dataset in datasets:
    print(f'\t{dataset.dataset_id}')

Datasets in project bruin-hiring:
	cdt_selcukemreozer
	github_repos


___

<h2>Question 1</h2>
1- What are the most popular repos?

In [31]:
# Creating table for q1
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q1 AS
    SELECT
      repo_name,
      COUNT(*) AS repo_counts
    FROM
      `bruin-hiring.github_repos.sample_commits`
    GROUP BY
      repo_name
    ORDER BY
      repo_counts DESC
    LIMIT 10;
    
"""

query_job = client.query(query)
print("For question 1, Table q1 has been created.")

For question 1, Table q1 has been created.


In [177]:
query = """
SELECT *
FROM
    bruin-hiring.cdt_selcukemreozer.q1
"""
query_job = client.query(query)
results = query_job.result()

x = [] # for chart, creating x and y axis lists
y = []

print("RESULTS:\n_______\n")

for row in results:
    x.append(row[0])
    y.append(row[1])
    print(row[0],'->' , row[1])

RESULTS:
_______

torvalds/linux -> 602604
apple/swift -> 38891
twbs/bootstrap -> 11777
Microsoft/vscode -> 6909
facebook/react -> 6875
tensorflow/tensorflow -> 5253


In [178]:
df = pd.DataFrame({'repo_name':x,'count':y})

In [179]:
# the Chart of question-1
alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field="count", type="quantitative"),
    color=alt.Color(field="repo_name", type="nominal"),
)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

___

<h2>Question 2</h2>
2- Who are the top contributors?

In [59]:
# Creating table for q2
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q2 AS
    SELECT 
      committer.name, 
      COUNT(*) as contribute_counts
    FROM 
      github_repos.sample_commits
    GROUP BY 
      committer.name
    ORDER BY 
      contribute_counts DESC
    LIMIT 10;
"""

query_job = client.query(query)

print("For question 2, Table q2 has been created.")

For question 2, Table q2 has been created.


In [163]:
query = """
SELECT
    *
FROM bruin-hiring.cdt_selcukemreozer.q2
"""

query_job = client.query(query)
results = query_job.result()

x2 = [] # for chart, creating x and y axis lists
y2 = []

print("RESULTS:\n_______\n")

for row in results:
    x2.append(row[0])
    y2.append(row[1])
    print(row[0],'->' , row[1])



RESULTS:
_______

Linus Torvalds -> 65331
Greg Kroah-Hartman -> 57661
David S. Miller -> 54669
Mauro Carvalho Chehab -> 23407
Ingo Molnar -> 19120
John W. Linville -> 18882
Mark Brown -> 17526
James Bottomley -> 9121
Takashi Iwai -> 8661
Russell King -> 8648


In [164]:
df2 = pd.DataFrame({'contributors':x2,'number of contribution':y2})

In [172]:
alt.Chart(df2).mark_bar().encode(x = 'contributors', y = 'number of contribution',
                                color = alt.condition(
                                alt.datum.contributors == 'Linus Torvalds',
                                alt.value('orange'),
                                alt.value('steelblue')
                                )).properties(width=400)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

___
<h2>Question 3</h2>
3- Are there any developer contributing to more than 1 repos?


In [70]:
# Creating table for q3
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q3 AS
    WITH developer_contribute as (
    SELECT
      committer.name as name,
      COUNT(DISTINCT repo_name) as repo_count
    FROM `github_repos.sample_commits`
    GROUP BY
      name
    )
    SELECT
      developer_contribute.name as name,
      developer_contribute.repo_count as repo_count
    FROM developer_contribute
    WHERE
      repo_count > 1;
"""

query_job = client.query(query)

print("For question 3, Table q3 has been created.")

For question 3, Table q3 has been created.


In [76]:
query = """
SELECT 
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q3;
"""
query_job = client.query(query)
results = query_job.result()


x3 = [] # for chart, creating x and y axis lists
y3 = []

print("RESULTS:\n_______\n")

for row in results:
    x3.append(row[0])
    y3.append(row[1])
    print(row[0],'->' , row[1])

RESULTS:
_______

Christian -> 2
Joshua Anderson -> 2
Julian Viereck -> 2
Richard -> 2
Jody McIntyre -> 2
Henry Wong -> 2
Daniel Mané -> 2
PatrickJS -> 2
Basarat Ali Syed -> 2
Steve Baker -> 2
James -> 2
Max F. Albrecht -> 2
Peter Blazejewicz -> 2
Alex -> 2
Pascal Borreli -> 2
Anton Blanchard -> 2
iamchenxin -> 2
Andrey Popp -> 2
Max -> 2
Kevin Robinson -> 2
Chris Rebert -> 2
Justas Brazauskas -> 2
Jack Zhang -> 2
James Friend -> 2
Greg Roodt -> 2
Baraa Hamodi -> 2
Prayag Verma -> 3
GitHub -> 4


In [158]:
df3 = pd.DataFrame({'name':x3,'number of diffirent repos':y3})

In [160]:
alt.Chart(df3).mark_bar().encode(x='name',y='number of diffirent repos')

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [182]:
print('Number of people who contribute repos more one:',len(x3))

Number of people who contribute repos more one: 28


___
<h2>Question 4</h2>
What are the most popular email domains?


In [89]:
# Creating table for q4
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q4 AS
    SELECT
      SUBSTR(committer.email, INSTR(committer.email, '@') + 1) AS domain,
      COUNT(*) AS domain_count
    FROM
      `github_repos.sample_commits`
    GROUP BY
      domain
    ORDER BY
      domain_count DESC
    LIMIT 10;
"""
query_job = client.query(query)

print("for q4, table q4 has been created")

for q4, table q4 has been created


In [91]:
query = """
SELECT
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q4;
"""

query_job = client.query(query)
results = query_job.result()

x4 = [] # for chart, creating x and y axis lists
y4 = []

print("RESULTS:\n_______\n")

for row in results:
    x4.append(row[0])
    y4.append(row[1])
    print(row[0],'->' , row[1])

RESULTS:
_______

davemloft.net -> 49523
redhat.com -> 44550
linux-foundation.org -> 40480
linuxfoundation.org -> 37969
apple.com -> 34030
suse.de -> 33295
intel.com -> 28992
gmail.com -> 24975
kernel.org -> 20221
tuxdriver.com -> 18875


In [156]:
df4 = pd.DataFrame({'domains':x4,'count':y4})

In [180]:
# alt.Chart(df4).mark_bar().encode(x='domains', y='count')
alt.Chart(df4).mark_arc().encode(
    theta=alt.Theta(field="count", type="quantitative"),
    color=alt.Color(field="domains", type="nominal"),
)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

___
<h2>Question 5</h2>
Number of commits per day



In [95]:
# Creating table for q5
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q5 AS
    SELECT 
      DATE(committer.date) as commit_dates,
      COUNT(*) AS commit_counts
    FROM 
      `github_repos.sample_commits`
    GROUP BY
      commit_dates
    ORDER BY
      commit_counts desc;
"""
query_job = client.query(query)

print("for q5, table q5 has been created")

for q5, table q5 has been created


In [129]:
query = """
SELECT 
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q5
    LIMIT 400;
"""

query_job = client.query(query)
results = query_job.result()

x5 = [] # for chart, creating x and y axis lists
y5 = []

print("RESULTS:\n_______\n")

for row in results:
    x5.append(row[0])
    y5.append(row[1])
    print(row[0],'->' , row[1])

RESULTS:
_______

2008-01-28 -> 1924
2007-10-10 -> 1424
2008-01-30 -> 1378
2008-01-25 -> 1148
2009-04-03 -> 1147
2008-04-17 -> 1133
2008-04-24 -> 939
2009-03-30 -> 938
2007-10-16 -> 928
2014-10-20 -> 913
2009-06-16 -> 900
2009-09-15 -> 874
2009-01-06 -> 867
2007-10-12 -> 866
2015-11-16 -> 852
2010-10-21 -> 843
2014-05-23 -> 840
2015-09-29 -> 812
2013-06-17 -> 755
2008-02-01 -> 746
2006-09-26 -> 738
2006-12-07 -> 738
2010-05-11 -> 736
2015-10-13 -> 716
2016-03-11 -> 703
2010-08-02 -> 692
2016-02-24 -> 690
2016-02-29 -> 686
2016-02-08 -> 682
2006-01-09 -> 681
2008-02-08 -> 673
2008-04-29 -> 664
2007-05-08 -> 660
2013-09-26 -> 650
2009-03-24 -> 650
2010-02-26 -> 638
2012-11-21 -> 633
2014-10-29 -> 627
2014-05-16 -> 624
2013-03-18 -> 622
2010-05-18 -> 616
2015-12-18 -> 613
2015-12-21 -> 612
2016-05-13 -> 606
2009-12-16 -> 606
2014-03-17 -> 603
2015-12-03 -> 602
2008-12-29 -> 602
2013-04-08 -> 602
2016-03-03 -> 600
2009-09-23 -> 599
2014-06-19 -> 596
2006-06-26 -> 596
2013-07-03 -> 594
2014

In [153]:
x5 = [str(date) for date in x5]
df5 = pd.DataFrame({'date':x5,'number of commits':y5})

In [154]:
alt.Chart(df5).mark_point().encode(x='date',y='number of commits')

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

___
<h2>Question 6</h2>
Number of commits per weekday

In [134]:
query = """
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q6 AS
    SELECT
      FORMAT_DATE('%A', committer.date) as weekdays,
      COUNT(*) as commit_numbers
    FROM `github_repos.sample_commits`
    GROUP BY 
      weekdays
    ORDER BY
      commit_numbers;
"""

query_job = client.query(query)
print("for q6, table q6 has been created")

for q6, table q6 has been created


In [140]:
query = """
SELECT
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q6
"""

query_job = client.query(query)
results = query_job.result()

x6 = list()
y6 = list()

print("RESULTS:\n_______\n")

for row in results:
    x6.append(row[0])
    y6.append(row[1])
    print(row[0],'->',row[1])

RESULTS:
_______

Sunday -> 43072
Saturday -> 44814
Friday -> 112297
Thursday -> 114669
Monday -> 115061
Wednesday -> 120722
Tuesday -> 121674


In [137]:
df6 = pd.DataFrame({'days':x6,'count':y6})

In [138]:
alt.Chart(df6).mark_point().encode(x = 'days', y = 'count')

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

___
<h2>Question 7</h2>
Any other insights you want to show<br>
- Number of commits which includes the word 'danger' for each repo<br>
- There is no any commits with 'danger' in vscode and tensorflow repos


In [139]:
query="""
CREATE TABLE bruin-hiring.cdt_selcukemreozer.q7 AS
    WITH danger_commits As (
    SELECT 
      committer.name as name,
      repo_name,
      subject
    FROM
     `github_repos.sample_commits`
    WHERE
     LOWER(subject) LIKE '%danger%'

    )
    SELECT 
      repo_name,
      COUNT(*) as counter
    FROM 
      danger_commits
    GROUP BY repo_name
    ORDER BY counter DESC;

"""
query_job = client.query(query)
print("for q7, table q7 has been created")

for q7, table q7 has been created


In [143]:
query="""
SELECT
    *
FROM
    bruin-hiring.cdt_selcukemreozer.q7
"""
query_job = client.query(query)
results = query_job.result()

x7 = list()
y7 = list()

print("RESULTS:\n_______\n")

for row in results:
    x7.append(row[0])
    y7.append(row[1])
    print(row[0],'->',row[1])


RESULTS:
_______

torvalds/linux -> 31
facebook/react -> 19
twbs/bootstrap -> 10
apple/swift -> 1


In [144]:
df7 = pd.DataFrame({'repo_name':x7,'count':y7})

In [149]:
alt.Chart(df7).mark_bar().encode(
    x='repo_name',
    y="count",
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.repo_name == 'torvalds/linux',  # If this test returns True,
        alt.value('orange'), # True
        alt.value('steelblue') # False
    )
).properties(width=300)

/Users/selcu/opt/anaconda3/lib/python3.10/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

_Selçuk Emre Özer_